# 튜토리얼 방법

이 튜토리얼에서는 요구 사항과 환경 변수가 워크샵의 모든 노트북에 저장되도록 이 초기 노트북을 먼저 실행해야 합니다.

## 시작 방법

1. 강사 주도 워크샵에 참석하거나 제공된 [CloudFormation 템플릿](https://raw.githubusercontent.com/aws-samples/prompt-engineering-with-anthropic-claude-v-3/main/cloudformation/workshop-v1-final-cfn.yml)을 사용하여 워크샵 인프라를 배포한 경우 2단계로 진행할 수 있습니다. 그렇지 않은 경우 워크샵 [GitHub 저장소](https://github.com/aws-samples/prompt-engineering-with-anthropic-claude-v-3)를 로컬 머신에 다운로드해야 합니다.

2. 다음 명령을 실행하여 필요한 종속성을 설치하세요.
 

> ⚠️ **pip의 종속성 해결 프로그램과 관련된 오류 메시지는 무시하세요.**

In [ ]:
%pip install -qU pip
%pip install -qUr requirements.txt --force-reinstall

3. 제공된 지침에 따라 노트북 셀을 순서대로 실행하세요.

---

## 사용 참고 사항 및 팁 💡

- 이 과정에서는 `temperature`는 0의 값을 가지고  Claude 3 Haiku를 사용합니다. `temperature` 에 대해서는 이 과정의 뒷부분에서 더 자세히 다룰 것입니다. 지금은 이러한 설정이 더 결정적인 결과를 산출한다는 것을 이해하면 됩니다. 이 과정의 모든 프롬프트 엔지니어링 기술은 Claude 2, Claude Instant 1.2와 같은 이전 세대 레거시 Claude 모델에도 적용됩니다.

- `Shift + Enter`를 사용하여 셀을 실행하고 다음 셀로 이동할 수 있습니다.

- 튜토리얼 페이지 하단에 도달하면 폴더 내 다음 번호가 매겨진 파일로 이동하거나, 해당 챕터 파일의 내용이 끝났다면 다음 번호가 매겨진 폴더로 이동하세요.

### Boto3 SDK 및 Converse API
이 튜토리얼 전체에서 [Amazon Boto3 SDK](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/bedrock-runtime.html)와 [Converse API](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/bedrock-runtime/client/converse.html)를 사용할 것입니다. 

아래는 이 튜토리얼에서 프롬프트를 실행하는 모습의 예시입니다. 먼저 `get_completion`이라는 헬퍼 함수를 만들어 프롬프트를 Claude에 보내고 Claude가 생성한 응답을 반환합니다. 지금 그 셀을 실행하세요.

먼저 모델 이름과 리전을 설정하고 저장합니다.

In [ ]:
import boto3
import json
from datetime import datetime
from botocore.exceptions import ClientError

session = boto3.Session()
region = session.region_name

In [ ]:
#modelId = 'anthropic.claude-3-sonnet-20240229-v1:0'
modelId = 'anthropic.claude-3-haiku-20240307-v1:0'

%store modelId
%store region

print(f'Using modelId: {modelId}')
print('Using region: ', region)

bedrock_client = boto3.client(service_name = 'bedrock-runtime', region_name = region,)

그런 다음 `get_completion`이라는 헬퍼 함수를 만들어 프롬프트를 Claude에 보내고 Claude가 생성한 응답을 반환합니다. 지금 그 셀을 실행하세요.

In [ ]:
def get_completion(prompt, system_prompt=None):
    # Define the inference configuration
    inference_config = {
        "temperature": 0.0,  # Set the temperature for generating diverse responses
        "maxTokens": 200  # Set the maximum number of tokens to generate
    }
    # Define additional model fields
    additional_model_fields = {
        "top_p": 1,  # Set the top_p value for nucleus sampling
    }
    # Create the converse method parameters
    converse_api_params = {
        "modelId": modelId,  # Specify the model ID to use
        "messages": [{"role": "user", "content": [{"text": prompt}]}],  # Provide the user's prompt
        "inferenceConfig": inference_config,  # Pass the inference configuration
        "additionalModelRequestFields": additional_model_fields  # Pass additional model fields
    }
    # Check if system_text is provided
    if system_prompt:
        # If system_text is provided, add the system parameter to the converse_params dictionary
        converse_api_params["system"] = [{"text": system_prompt}]

    # Send a request to the Bedrock client to generate a response
    try:
        response = bedrock_client.converse(**converse_api_params)

        # Extract the generated text content from the response
        text_content = response['output']['message']['content'][0]['text']

        # Return the generated text content
        return text_content

    except ClientError as err:
        message = err.response['Error']['Message']
        print(f"A client error occured: {message}")

이제 Claude에 대한 예시 프롬프트를 작성하고 `get_completion` 헬퍼 함수를 실행하여 Claude의 출력을 인쇄할 것입니다. 아래 셀을 실행하면 그 아래에 Claude의 응답이 출력됩니다.

프롬프트 문자열을 수정하여 Claude에게 다른 응답을 이끌어 내는 것도 해보세요.

In [ ]:
# Prompt
prompt = "Hello, Claude!"

# Get Claude's response
print(get_completion(prompt))

앞서 정의한 `modelId`와 `region` 변수는 이 튜토리얼 전체에서 사용됩니다. 각 튜토리얼 페이지의 셀을 위에서부터 아래로 실행하는 것만 확인하세요.